In [24]:
from imports import *
from data import *
from arch import *
model = return_token_model(CORE_MODEL, len(labels), preferred_cuda_test);
#model.load_state_dict(torch.load(CHECKPOINT_PATH + ROOT_NAME + '3_17000.pt')["model_state_dict"]);
#model.load_state_dict(torch.load("/mnt/SSD2/pholur/CTs/checkpoints/Day_0928_Insider_Outsider_79_54480.pt")["model_state_dict"]);
model.load_state_dict(torch.load("/mnt/SSD2/pholur/CTs/checkpoints/Day_0928_Insider_Outsider_20_14301.pt")["model_state_dict"]);
model.eval();

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able

In [25]:
from shared_train_and_test_functions import tokenization
import spacy
from spacy import displacy
import nltk
nlp = spacy.load('en_core_web_sm')
# ONLY WORKS FOR SINGLE POST TESTING

In [26]:
def return_insiders_and_outsiders(text):
    doc = nlp(text)
    add_np_text = []
    spans = []
    for npy in doc.noun_chunks:
        add_np_text.append(npy.text)
        spans.append((npy.start_char, npy.end_char))

    test_encodings, tokens = tokenization([text], True)
    offset_mappings = test_encodings.offset_mapping[0]

    input_ids = torch.tensor(test_encodings['input_ids']).to(preferred_cuda_test)
    attention_mask = torch.tensor(test_encodings['attention_mask']).to(preferred_cuda_test)
    outputs = model(input_ids, attention_mask=attention_mask) # try this: labels = torch.tensor([0]*len(input_ids)).to(preferred_cuda_test))
    predictions = outputs[0].detach().cpu().numpy()
    single_prediction_post = predictions[0]
    predictions_argmax = np.argmax(single_prediction_post, axis=1)
    print(predictions_argmax)
    ents = []
    total_true = {}

    for i in range(len(spans)):
        total_true[i] = False
    
    for j in range(len(spans)):
        
        choice = "no"
        for i,index in enumerate(predictions_argmax):
            current_token_mapping = offset_mappings[i]
 
            if current_token_mapping[0] >= spans[j][0] and current_token_mapping[1] <= spans[j][1]: # if the token is in the span of a chunk
                if index == 2:
                    choice = "yes"
                else:
                    choice = "no"

        if choice == "no":
            ents.append({"start": spans[j][0], "end": spans[j][1], "label": "INSIDER"})
        else:
            ents.append({"start": spans[j][0], "end": spans[j][1], "label": "OUTSIDER"})

    # deduplicate list of dictionaries
    ents = [dict(t) for t in {tuple(d.items()) for d in ents}]
    # sort list of dictionaries by start index
    ents = sorted(ents, key=lambda x: x['start'])
    print(ents)
    options = {"colors": {"INSIDER":"#88C6F1", "OUTSIDER":"#FF82AB"}}
    ex = [{"text": text, "ents": ents}]
    displacy.render(ex, style="ent", manual=True, options=options)

In [27]:
#posts = ["I think that #BigTech will kill me with vaccines. People like Bill Gates are doing it for the money. My friend, Sarah, who is a doctor, told me not to get the vaccine, because it causes cancer."]
posts = ["5g causes cancer", "5g is good for the economy.", "We should burn cell phone towers.", "We should build cell phone towers.", "Radio waves are as potent as vaccines.", "Vaccines are not harmful.", "Microchips are telling the government where we are 24/7.", "Vaccines contain microchips.", \
    ">doesn't make sense how is wanting safety for a new experimental drug not make sense? even if you got the vaccine you still run risk to get covid, need to isolate and wear a mask, and still have the same risk of death. the company even hid any side effects and threatened to sue anyone mentioning side effects. the people taking the vaccine now are just part of the testing trials, they are volunteer test subjects, since we don't even know long term effects or any effect past 1 year of taking it"]
#posts = ["I think that #BigTech will save me with vaccines. People like Bill Gates are doing it to save people. My friend, Sarah, who is a doctor, told me to get the vaccine, because it does not causes small pox."]
for p in posts:
    return_insiders_and_outsiders(p)

[2 2 2 2 2 2]
[{'start': 0, 'end': 2, 'label': 'OUTSIDER'}, {'start': 10, 'end': 16, 'label': 'OUTSIDER'}]


[2 0 2 0 0 0 2 0 2 0]
[{'start': 0, 'end': 2, 'label': 'INSIDER'}, {'start': 15, 'end': 26, 'label': 'INSIDER'}]


[2 0 0 0 2 2 2 2 0]
[{'start': 0, 'end': 2, 'label': 'INSIDER'}, {'start': 15, 'end': 32, 'label': 'OUTSIDER'}]


[2 0 0 0 0 0 0 2 0]
[{'start': 0, 'end': 2, 'label': 'INSIDER'}, {'start': 16, 'end': 33, 'label': 'INSIDER'}]


[2 2 2 2 2 2 2 2 2 2]
[{'start': 0, 'end': 11, 'label': 'OUTSIDER'}, {'start': 29, 'end': 37, 'label': 'OUTSIDER'}]


[2 2 2 2 0 2 2 2 0]
[{'start': 0, 'end': 8, 'label': 'INSIDER'}]


[2 2 2 2 0 2 2 2 2 0 0 2 2 2 2 2]
[{'start': 0, 'end': 10, 'label': 'OUTSIDER'}, {'start': 23, 'end': 37, 'label': 'OUTSIDER'}, {'start': 44, 'end': 46, 'label': 'INSIDER'}]


[2 2 2 2 2 2 2 2 2 0]
[{'start': 0, 'end': 8, 'label': 'INSIDER'}, {'start': 17, 'end': 27, 'label': 'OUTSIDER'}]


[2 2 2 2 2 2 2 0 2 0 2 2 2 2 2 2 2 2 2 0 0 1 0 2 2 1 2 2 2 2 2 2 2 2 2 2 0
 0 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 2 2 0 0
 0 1 1 2 0 0 2 2 0 0 0 0 2 0 0 0 0 0 0 2 2 2 2 2 2 0 0 0 2 0 0 2]
[{'start': 14, 'end': 19, 'label': 'OUTSIDER'}, {'start': 35, 'end': 41, 'label': 'INSIDER'}, {'start': 46, 'end': 69, 'label': 'OUTSIDER'}, {'start': 79, 'end': 84, 'label': 'OUTSIDER'}, {'start': 94, 'end': 97, 'label': 'INSIDER'}, {'start': 102, 'end': 113, 'label': 'OUTSIDER'}, {'start': 114, 'end': 117, 'label': 'INSIDER'}, {'start': 128, 'end': 132, 'label': 'OUTSIDER'}, {'start': 140, 'end': 145, 'label': 'OUTSIDER'}, {'start': 172, 'end': 178, 'label': 'OUTSIDER'}, {'start': 195, 'end': 208, 'label': 'OUTSIDER'}, {'start': 212, 'end': 217, 'label': 'OUTSIDER'}, {'start': 219, 'end': 230, 'label': 'OUTSIDER'}, {'start': 240, 'end': 256, 'label': 'OUTSIDER'}, {'start': 279, 'end': 285, 'label': 'OUTSIDER'}, {'start': 297, 'end': 309, 'label': 'OUTSIDER'}, {'start': 31

In [28]:
test_loss = 0
number_of_test_batches = 0
test_loader = get_data(INPUT_DATA_PATH, FLAG, AUG, REEXTRACT, FRACTION, "test")

for j,test_batch in enumerate(test_loader):
    input_ids = test_batch['input_ids'].to(preferred_cuda_test)
    attention_mask = test_batch['attention_mask'].to(preferred_cuda_test)
    labels = test_batch['labels'].to(preferred_cuda_test)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        test_loss += outputs[0]
    number_of_test_batches += 1
print("Total test loss:", test_loss/number_of_test_batches)

Using separate samples per entity: Size =  648
Total test loss: tensor(0.8864, device='cuda:1')
